In [49]:
#4 satellites Doppler 1 frequency

#Solving for pose and frequency offset

#Rotation of Earth and the effect of the ionosphere are included

#Doppler Method for Localizing a receiver
#Fausto Vega

using SatelliteDynamics
using LinearAlgebra
using PlotlyJS
using DelimitedFiles
using Distributions

In [50]:
# Declare simulation initial Epoch. Start time
epc0 = Epoch(2019, 1, 1, 12, 0, 0, 0.0)  #year, month, day, hour, minute, seconds, nanoseconds 

Epoch(2019-01-01T11:59:23.000Z)

In [51]:
# Declare initial state in terms of osculating orbital elements
#semi-major axis, eccentricity, inclination, RAAN, Argument of perigee, Mean anomaly

#1 and 2 on the same orbit, different initial mean anomaly

#working for this orbit
#iss1 = [7378e3, 0.0004879, 90.6391, 189.0859, 151.2014, 185]; #converges at 189.0859
#iss2 = [7378e3, 0.0004879, 90.6391, 189.0859, 151.2014, 205];
#iss3 = [7378e3, 0.0004879, 90.6391, 201.0859, 151.2014, 190]; # converges a t 200.0859
#iss4 = [7378e3, 0.0004879, 90.6391, 201.0859, 151.2014, 210]; # converges a t 200.0859


#ARGOS Orbit ~850 km altitude

iss1 = [7221e3, 0.0004879, 90.6391, 194.5859, 151.2014, 190]; 
iss2 = [7221e3, 0.0004879, 90.6391, 194.5859, 151.2014, 200];
iss3 = [7221e3, 0.0004879, 90.6391, 195.5859, 151.2014, 193]; 
iss4 = [7221e3, 0.0004879, 90.6391, 195.5859, 151.2014, 203]; 


#Function to convert mean anomoaly to eccentric anomaly
E1 = anomaly_mean_to_eccentric(iss1[6], iss1[2], use_degrees=true); 
E2 = anomaly_mean_to_eccentric(iss2[6], iss2[2], use_degrees=true);
E3 = anomaly_mean_to_eccentric(iss3[6], iss3[2], use_degrees=true);
E4 = anomaly_mean_to_eccentric(iss4[6], iss4[2], use_degrees=true);

In [52]:
# Convert osculating elements to Cartesean state
# returns position and velocity (m, m/s)

eci0_1 = sOSCtoCART(iss1, use_degrees=true)
eci0_2 = sOSCtoCART(iss2, use_degrees=true)
eci0_3 = sOSCtoCART(iss3, use_degrees=true)
eci0_4 = sOSCtoCART(iss4, use_degrees=true)

6-element Vector{Float64}:
      -6.920528693041507e6
      -1.9388943140103715e6
 -732572.827379684
    -746.2067396656779
    -122.5914937915185
    7387.743541399651

In [53]:
#compute the satellite orbital period given the semi-major axis

#same for all 4 satellites since they have the same semi-major axis
T = orbit_period(iss1[1])

#final time for one orbit. Adds initial time to the orbital period

epcf = epc0 + T

Epoch(2019-01-01T13:41:09.705Z)

In [54]:
# Create an EarthInertialState orbit propagagator
#needs initial epoch of state and the state vector
orb1  = EarthInertialState(epc0, eci0_1, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb2  = EarthInertialState(epc0, eci0_2, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb3  = EarthInertialState(epc0, eci0_3, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

orb4  = EarthInertialState(epc0, eci0_4, dt=1.0,
            mass=1.0, n_grav=0, m_grav=0,
            drag=false, srp=false,
            moon=false, sun=false,
            relativity=false
)

# Propagate the orbit
# orbit until the final time

t_1, epc_1, eci_1 = sim!(orb1, epcf);
t_2, epc_2, eci_2 = sim!(orb2, epcf);
t_3, epc_3, eci_3 = sim!(orb3, epcf);
t_4, epc_4, eci_4 = sim!(orb4, epcf);

In [55]:
#equator position in cartesian coordinates
tag =[-6.128804e6, -1.590206e6, 776.1502e3]

#Examples of putting latitude and longitude and converting to ECEF coordinate frame
#Equitorial position
#tag_geof = [-165.4545, 16.98849, 0]

#North pole position
#tag_geof = [-165.4545, 46.98849, 0]

#tag = sGEOCtoECEF(tag_geof, use_degrees = true)

3-element Vector{Float64}:
     -6.128804e6
     -1.590206e6
 776150.2

In [56]:
#Equitorial Position
#In m
x0= tag[1] 
y0 = tag[2]
z0 = tag[3]

#λ0 = C_LIGHT/400e6 #nominal wavelength in m

f0 = 1e6

#r0 = [x0, y0, z0, λ0]
r0 = [x0, y0, z0, f0]

centroid_guess = [(eci_1[1,1]+eci_2[1,1]+eci_3[1,1]+eci_4[1,1])/4, (eci_1[2,1]+eci_2[2,1]+eci_3[2,1]+eci_4[2,1])/4, (eci_1[3,1]+eci_2[3,1]+eci_3[3,1]+eci_4[3,1])/4] 
onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
geodetic = [onearth[1], onearth[2], 0]

#Guess
xyz = sGEOCtoECEF(geodetic, use_degrees = true)

println(xyz)

[-6.012592823500298e6, -1.636770880711428e6, -1.3602721101371013e6]


In [57]:
#Plot Satellite 1 and 2 Orbit
sat1 = scatter(x=eci_1[1,:], y=eci_1[2,:], z=eci_1[3,:], type="scatter3d", mode="lines", name="orbit 1&2")

#Plot Satellite 3 and 4 Orbit
sat3 = scatter(x=eci_3[1,:], y=eci_3[2,:], z=eci_3[3,:], type="scatter3d", mode="lines", name="orbit 3&4")

#Plot all 4 satellite initial positions
satellites = scatter(x=[eci_1[1,1],eci_2[1,1], eci_3[1,1], eci_4[1,1]], y=[eci_1[2,1],eci_2[2,1], eci_3[2,1], eci_4[2,1]],z=[eci_1[3,1],eci_2[3,1], eci_3[3,1], eci_4[3,1]], mode="markers", marker_size = 4, type="scatter3d", name="satellites")

#Plot Tag position
tag = scatter(x = [r0[1], r0[1]], y = [r0[2], r0[2]], z = [r0[3], r0[3]], type="scatter3d", name="tag", mode="markers", marker_size=5)

#Plot the Guess
guess = scatter(x = [xyz[1], xyz[1]], y = [xyz[2], xyz[2]], z = [xyz[3], xyz[3]], type="scatter3d", name="guess", mode="markers", marker_size=5)

#Plotting the sphere (Earth)
n = 100
u = range(-π, π; length = n)
v = range(0, π; length = n)
x = cos.(u) * sin.(v)'
y = sin.(u) * sin.(v)' 
z = ones(n) * cos.(v)' 

earth = surface(z=z*6371000, x=x*6371000, y=y*6371000)

#plot([sat1,sat3, satellites, tag, earth, guess]) #uncomment to see the plot

surface with fields type, x, y, and z


In [58]:
#Finding the Zenith Angle

function zenith_angle(satposes, r0)
    
    zenithangle = zeros(4)

    #vector between tag and the satellite
    normalvec = r0
    
    for i in 1:4
        
        vector = satposes[i,:] - r0[1:3]
        
        #Find the angle between the normal and the vector going from the tag to the satellte
        theta = acos(dot(normalvec,vector)/(norm(normalvec)*norm(vector))) * (180/pi)
        
        #save zenith angle
        zenithangle[i] = theta
    end
    
    return zenithangle
end

zenith_angle (generic function with 1 method)

In [59]:
#Testing zenith angle function
satposes = [eci_1[1,1] eci_1[2,1] eci_1[3,1];eci_2[1,1] eci_2[2,1] eci_2[3,1];eci_3[1,1] eci_3[2,1] eci_3[3,1];eci_4[1,1] eci_4[2,1] eci_4[3,1]]

zenith_angles = zenith_angle(satposes, r0[1:3])

println(zenith_angles)

#check all angles to see if it is on the horizon 
inhorizon = all(x->x<80, zenith_angles)

[87.71661886808683, 73.7916614769131, 84.30998334955072, 67.55471562049871]


false

In [60]:
#Obtain the orbital elements after each time step (not used in the sim) 
dim = size(t_1)[1]

all_elements = zeros(6,dim)
for i in 1:dim
    
    orbital_elements = sCARTtoOSC(eci_1[:,1], use_degrees=true)
    all_elements[:,i] = orbital_elements
end 

In [61]:
#velocity is the last 3 elements of each column in eci matrices
eci_1[:,1]

6-element Vector{Float64}:
    -6.611755461229845e6
    -1.7473412825817908e6
    -2.3290448551643165e6
 -2336.2081282888425
  -526.900136186714
  7029.342674382558

In [62]:
#Generate TEC Distribution
mu = 8e-4
sigma = 3e-4
lb = 3e-4
ub = 13e-4
d = Truncated(Normal(mu,sigma), lb, ub)

#TEC1 = 3e-4
#TEC2 = 4e-4
#TEC3 = 3.5e-4
#TEC4 = 4.5e-4

#f = 1575.42e6 # in 1/s L1 frequency
f = 400e6 # in 1/s system frequency
h = 350e3 #in meters
Ip = zeros(4)
OF = zeros(4)
Iz = zeros(4)
Ip_scaled = zeros(4)

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [63]:
using ForwardDiff
using BlockDiagonals

distance_scale = R_EARTH*(1e-3)*1e-4 # scales km to custom scale working

time_scale = 1/(C_LIGHT/R_EARTH) #scales seconds to custom scale

velocity_scale = distance_scale/time_scale #scales velocities to custom scale

#frequency_scale = 1e2 #to improve the numerical conditioning of the matrix working for f0= 1e6
frequency_scale = 1e5 #working for f0 = 400e6


#frequency_scale = 1/time_scale #higher conditioning ~70 condition num compared to 15

c = 10000 #km/s

10000

In [64]:
1/distance_scale

1.567856114959475

In [65]:
velocity_scale

29.9792458

In [66]:
#satposet is all the satellite known positions and the measured position
#x is what we want to solve for
#satpose is the satellite position and velocities as well as the measured change in frequency

function doppler_residual(x, sat_pose_f, time)
    
    omega = OMEGA_EARTH*time_scale #transform to custom scale
    
    #wavelength = 0.7494*1e-3/distance_scale #for a nominal frequency of 400 MHz
    
    omegahat = [0 -omega 0; omega 0 0; 0 0 0]
    
    A1 = [cos(omega*time[1]) sin(omega*time[1]) 0;
          -sin(omega*time[1]) cos(omega*time[1]) 0;
          0 0 1];
    A2 = [cos(omega*time[2]) sin(omega*time[2]) 0;
          -sin(omega*time[2]) cos(omega*time[2]) 0;
          0 0 1];
    A3 = [cos(omega*time[3]) sin(omega*time[3]) 0;
          -sin(omega*time[3]) cos(omega*time[3]) 0;
          0 0 1];
    A4 = [cos(omega*time[4]) sin(omega*time[4]) 0;
          -sin(omega*time[4]) cos(omega*time[4]) 0;
          0 0 1];
    
    res1 = (x[4]/c)*0.5*(1/norm(x[1:3] - A1*sat_pose_f[1,1:3]))*(-2*x[1:3]'*A1*omegahat*sat_pose_f[1,1:3] - 2*x[1:3]'*A1*sat_pose_f[1,4:6]+sat_pose_f[1,1:3]'*sat_pose_f[1,4:6]+sat_pose_f[1,4:6]'*sat_pose_f[1,1:3])- sat_pose_f[1,7]
    res2 = (x[4]/c)*0.5*(1/norm(x[1:3] - A2*sat_pose_f[2,1:3]))*(-2*x[1:3]'*A2*omegahat*sat_pose_f[2,1:3] - 2*x[1:3]'*A2*sat_pose_f[2,4:6]+sat_pose_f[2,1:3]'*sat_pose_f[2,4:6]+sat_pose_f[2,4:6]'*sat_pose_f[2,1:3])- sat_pose_f[2,7]
    res3 = (x[4]/c)*0.5*(1/norm(x[1:3] - A3*sat_pose_f[3,1:3]))*(-2*x[1:3]'*A3*omegahat*sat_pose_f[3,1:3] - 2*x[1:3]'*A3*sat_pose_f[3,4:6]+sat_pose_f[3,1:3]'*sat_pose_f[3,4:6]+sat_pose_f[3,4:6]'*sat_pose_f[3,1:3])- sat_pose_f[3,7]
    res4 = (x[4]/c)*0.5*(1/norm(x[1:3] - A4*sat_pose_f[4,1:3]))*(-2*x[1:3]'*A4*omegahat*sat_pose_f[4,1:3] - 2*x[1:3]'*A4*sat_pose_f[4,4:6]+sat_pose_f[4,1:3]'*sat_pose_f[4,4:6]+sat_pose_f[4,4:6]'*sat_pose_f[4,1:3])- sat_pose_f[4,7]
    
    return [res1; res2; res3; res4]
   
end

doppler_residual (generic function with 1 method)

In [67]:
#pose1-4 are satellite locations and velocities
#r0 = [xyz of tag, emmitted frequency]

function measurment(r0, pose1, pose2, pose3, pose4, time)
    
    #Make sure to scale all positions
    
    omega = OMEGA_EARTH*time_scale #change to custom time scale
    
    #wavelength = 0.7494*1e-3/distance_scale #for a nominal frequency of 400 MHz
    
    omegahat = [0 -omega 0; omega 0 0; 0 0 0]
    
    A1 = [cos(omega*time[1]) sin(omega*time[1]) 0;
          -sin(omega*time[1]) cos(omega*time[1]) 0;
          0 0 1];
    A2 = [cos(omega*time[2]) sin(omega*time[2]) 0;
          -sin(omega*time[2]) cos(omega*time[2]) 0;
          0 0 1];
    A3 = [cos(omega*time[3]) sin(omega*time[3]) 0;
          -sin(omega*time[3]) cos(omega*time[3]) 0;
          0 0 1];
    A4 = [cos(omega*time[4]) sin(omega*time[4]) 0;
          -sin(omega*time[4]) cos(omega*time[4]) 0;
          0 0 1];
    
    deltaf1 = (r0[4]/c)*0.5*(1/norm(r0[1:3] - A1*pose1[1:3]))*(-2*r0[1:3]'*A1*omegahat*pose1[1:3] - 2*r0[1:3]'*A1*pose1[4:6]+pose1[1:3]'*pose1[4:6]+pose1[4:6]'*pose1[1:3])
    deltaf2 = (r0[4]/c)*0.5*(1/norm(r0[1:3] - A2*pose2[1:3]))*(-2*r0[1:3]'*A2*omegahat*pose2[1:3] - 2*r0[1:3]'*A2*pose2[4:6]+pose2[1:3]'*pose2[4:6]+pose2[4:6]'*pose2[1:3])
    deltaf3 = (r0[4]/c)*0.5*(1/norm(r0[1:3] - A3*pose3[1:3]))*(-2*r0[1:3]'*A3*omegahat*pose3[1:3] - 2*r0[1:3]'*A3*pose3[4:6]+pose3[1:3]'*pose3[4:6]+pose3[4:6]'*pose3[1:3])
    deltaf4 = (r0[4]/c)*0.5*(1/norm(r0[1:3] - A4*pose4[1:3]))*(-2*r0[1:3]'*A4*omegahat*pose4[1:3] - 2*r0[1:3]'*A4*pose4[4:6]+pose4[1:3]'*pose4[4:6]+pose4[4:6]'*pose4[1:3])

    return [deltaf1, deltaf2, deltaf3, deltaf4]
end

measurment (generic function with 1 method)

In [68]:
#Equitorial Position

#In km
x0= x0*1e-3 #x tag position
y0 = y0*1e-3 #y tag position
z0 = z0*1e-3 #z tag position
#f0 = 1e6 #nominal frequency (works)
f0 = 400e6

r0_scaled = [x0/distance_scale, y0/distance_scale, z0/distance_scale, f0/frequency_scale]

4-element Vector{Float64}:
 -9609.08282878809
 -2493.214201145247
  1216.8918371970192
  4000.0

In [69]:
diff = 50e3*1e-3/distance_scale 

78.39280574797374

In [70]:
function receiver_pose(all_sats_scaled, guess, zenith_angles, time, zdot) # remember to scale the guess to custom units

    n = 1000 # number of iterations
    #all_r0 = NaN*[zeros(4) for i = 1:n]
    all_r0 = [zeros(3) for i = 1:n] #obtain all the tag positions
    
    sat1poses = zeros(7,n)
    sat2poses = zeros(7,n)
    sat3poses = zeros(7,n)
    sat4poses = zeros(7,n)

    all_sats_noise = zeros(4,4)

    iters = 0

     #Monte Carlo Simulation
    for i in 1:n
        
        # Testing Parameters for line search
        #b = 0.1
        #c_ =0.5
        #β = 1.0
        #α = 1
        
        
        # working Parameters for line search
        b = 0.01
        c_ =0.5
        β = 10
        α = 1
        
        #create noise from normal distribution
        #scaled to the variable (0.1m error for distance & 1e-11 for time)
        gpsnoise = randn(12)*(0.1*1e-3/distance_scale) #0.1 m switch to km then to custom scale
        velocitynoise = randn(12)*(0.01*1e-3/velocity_scale)# 0.1 m/s
        doppler_noise = randn(12)* 1/frequency_scale #0.001 ~ 500's of error
        #clocknoise = randn(4)*(1e-10/time_scale)
        
        TEC = rand(d,4) #vTEC for each satellite from custom distribution
        
        #added ionosphere noise
        Idot1 = (((40.3*TEC[1])*((R_EARTH*sind(zenith_angles[1]))*(R_EARTH*cosd(zenith_angles[1])*zdot[1])/(R_EARTH + h)^2))/((f^2)*(1-((R_EARTH*sind(zenith_angles[1]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale
        Idot2 = (((40.3*TEC[2])*((R_EARTH*sind(zenith_angles[2]))*(R_EARTH*cosd(zenith_angles[2])*zdot[2])/(R_EARTH + h)^2))/((f^2)*(1-((R_EARTH*sind(zenith_angles[2]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale
        Idot3 = (((40.3*TEC[3])*((R_EARTH*sind(zenith_angles[3]))*(R_EARTH*cosd(zenith_angles[3])*zdot[3])/(R_EARTH + h)^2))/((f^2)*(1-((R_EARTH*sind(zenith_angles[3]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale
        Idot4 = (((40.3*TEC[4])*((R_EARTH*sind(zenith_angles[4]))*(R_EARTH*cosd(zenith_angles[4])*zdot[4])/(R_EARTH + h)^2))/((f^2)*(1-((R_EARTH*sind(zenith_angles[4]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale
        
        iono1 = ((f0/frequency_scale)/c)*Idot1
        iono2 = ((f0/frequency_scale)/c)*Idot2
        iono3 = ((f0/frequency_scale)/c)*Idot3
        iono4 = ((f0/frequency_scale)/c)*Idot4
        
        #println("this is ionosphere error: ", iono1*frequency_scale) #arond 3 Hz
        
        #println("this is zdot: ", zdot)
        #println("this is ionosphere error: ", iono2*frequency_scale)
        #println("this is ionosphere error: ", iono3*frequency_scale)
        #println("this is ionosphere error: ", iono4*frequency_scale)
        
        
        # 
        #Calculating random TEC time delay
        #for i=1:4
        
        #    OF[i] = (1-((R_EARTH*sind(zenith_angles[i]))/(R_EARTH+hi))^2)^(-1/2) #normal units (m and s)
        #    Ip[i] = ((40.3*TEC[i])/(f^2))*OF[i] * 1e-3 #scale to km to use the custom unit
        #    Ip_scaled[i] = Ip[i]/distance_scale #scale to custom units
        
        #end
        
        sat1_noise = [all_sats_scaled[1,1]+gpsnoise[1],all_sats_scaled[1,2] + gpsnoise[2],all_sats_scaled[1,3]+gpsnoise[3], all_sats_scaled[1,4]+velocitynoise[1], all_sats_scaled[1,5]+velocitynoise[2],all_sats_scaled[1,6]+velocitynoise[3], all_sats_scaled[1,7] + iono1 + doppler_noise[1]]
        sat2_noise =[all_sats_scaled[2,1]+gpsnoise[4],all_sats_scaled[2,2] + gpsnoise[5],all_sats_scaled[2,3]+gpsnoise[6], all_sats_scaled[2,4]+velocitynoise[4], all_sats_scaled[2,5]+velocitynoise[5],all_sats_scaled[2,6]+velocitynoise[6], all_sats_scaled[2,7] + iono2 + doppler_noise[2]]
        sat3_noise = [all_sats_scaled[3,1]+gpsnoise[7],all_sats_scaled[3,2] + gpsnoise[8],all_sats_scaled[3,3]+gpsnoise[9], all_sats_scaled[3,4]+velocitynoise[7], all_sats_scaled[3,5]+velocitynoise[8],all_sats_scaled[3,6]+velocitynoise[9], all_sats_scaled[3,7] + iono3 + doppler_noise[3]]
        sat4_noise = [all_sats_scaled[4,1]+gpsnoise[10],all_sats_scaled[4,2] + gpsnoise[11],all_sats_scaled[4,3]+gpsnoise[12], all_sats_scaled[4,4]+velocitynoise[10], all_sats_scaled[4,5]+velocitynoise[11],all_sats_scaled[4,6]+velocitynoise[12], all_sats_scaled[4,7] + iono4 + doppler_noise[4]]

        all_sats_noise = vcat(sat1_noise',sat2_noise',sat3_noise',sat4_noise')
        
        sat1poses[:,i] = sat1_noise
        sat2poses[:,i] = sat2_noise
        sat3poses[:,i] = sat3_noise
        sat4poses[:,i] = sat4_noise
        
        X = NaN*[zeros(4) for i = 1:n]
        R = NaN*[zeros(4) for i = 1:n]
        
        #Centroid guess
#         X[1] = [guess[1]/distance_scale, guess[2]/distance_scale, guess[3]/distance_scale, 0.75e6/frequency_scale]
        X[1] = [r0_scaled[1]+diff, r0_scaled[2]+diff, r0_scaled[3]+diff, 410e6/frequency_scale]

        #X[1] = r0_scaled
        #working initial guess without ionosphere effects
        #X[1] = [guess[1]/distance_scale, guess[2]/distance_scale, guess[3]/distance_scale, 410e6/frequency_scale]
        #println("intial guess: ", X[1])
        #Initial residual
        R[1] = doppler_residual(X[1], all_sats_noise, time)

        iters = 0

        for k=1:1000
            R[k] = doppler_residual(X[k], all_sats_noise, time) #calculate residual

            
            #println("this is the residual: ", norm(R[k]))
            iters += 1

            #if(norm(R[k]) < 1e-12)
            
            if(norm(R[k]) < 1e-6)
                
                break

            end

            jacobian = ForwardDiff.jacobian(dx -> doppler_residual(dx, all_sats_noise, time), X[k])
            
            conditionnum = cond(jacobian)
            
            if iters > 50
                break
            end
            
            #println("this is the condition number: ", conditionnum)
            
            #println("this is the jacobian: ", jacobian)

            deltax = (jacobian)\-R[k]
            
            while norm(doppler_residual(X[k] + α*deltax, all_sats_noise, time)) > norm(doppler_residual(X[k], all_sats_noise, time) + b*α*jacobian'*deltax)

                α = c_*α
                #print("this is alpha: ", α)
            end

            X[k+1] = X[k] + α*deltax
             
        end  
        
        R = R[1:iters]
        X = X[1:iters]

        all_r0[i] = X[end]

    end
    x_values = zeros(n)
    y_values = zeros(n)
    z_values = zeros(n)

    for j in 1:n

        x_values[j] = all_r0[j][1]
        y_values[j] = all_r0[j][2]
        z_values[j] = all_r0[j][3]

    end
    
    #mean calculation
    mean = sum(all_r0, dims = 1)/ n
    mean = mean[1]
    mean_rescaled = [mean[1:3]*distance_scale*1e3; mean[4]*frequency_scale] # in meters
    
    return mean_rescaled, all_r0, iters
end

receiver_pose (generic function with 1 method)

In [71]:
#Navigation
#count = 0
n=1000
PDOP_array = zeros(size(eci_1)[2],1)

for i=1:size(eci_1)[2] - 1
    
    #original working
    #satposes = [eci_1[1,i] eci_1[2,i] eci_1[3,i];eci_2[1,i] eci_2[2,i] eci_2[3,i];eci_3[1,i] eci_3[2,i] eci_3[3,i];eci_4[1,i] eci_4[2,i] eci_4[3,i]]
    #zenith_angles = zenith_angle(satposes, r0[1:3])
    
    satposes = [eci_1[1,i] eci_1[2,i] eci_1[3,i];eci_2[1,i] eci_2[2,i] eci_2[3,i];eci_3[1,i] eci_3[2,i] eci_3[3,i];eci_4[1,i] eci_4[2,i] eci_4[3,i]]
    satposes2 = [eci_1[1,i+1] eci_1[2,i+1] eci_1[3,i+1];eci_2[1,i+1] eci_2[2,i+1] eci_2[3,i+1];eci_3[1,i+1] eci_3[2,i+1] eci_3[3,i+1];eci_4[1,i+1] eci_4[2,i+1] eci_4[3,i+1]]
    zenith_angles = zenith_angle(satposes, r0[1:3])
    zenith_angles2 = zenith_angle(satposes2, r0[1:3])
    
    zdot = zenith_angles - zenith_angles2
    
    #zenith_angles = zenith_angle(satposes, r0[1:3])

    #check all angles to see if it is on the horizon 
    
    inhorizon = all(x->x<70, zenith_angles)
    
    if inhorizon==false
        
        continue #skip the current iteration
    end

    #In km then to custom units
    r1 = [eci_1[1,i], eci_1[2,i], eci_1[3,i]]*1e-3/distance_scale
    r2 = [eci_2[1,i], eci_2[2,i], eci_2[3,i]]*1e-3/distance_scale
    r3 = [eci_3[1,i], eci_3[2,i], eci_3[3,i]]*1e-3/distance_scale
    r4 = [eci_4[1,i], eci_4[2,i], eci_4[3,i]]*1e-3/distance_scale
    
    v1 = [eci_1[4,i], eci_1[5,i], eci_1[6,i]]*1e-3/velocity_scale
    v2 = [eci_2[4,i], eci_2[5,i], eci_2[6,i]]*1e-3/velocity_scale
    v3 = [eci_3[4,i], eci_3[5,i], eci_3[6,i]]*1e-3/velocity_scale
    v4 = [eci_4[4,i], eci_4[5,i], eci_4[6,i]]*1e-3/velocity_scale

    pose1 = [r1' v1']
    pose2 = [r2' v2']
    pose3 = [r3' v3']
    pose4 = [r4' v4']
    
    time = [0.006, 0.006, 0.006, 0.008]/time_scale # assume a fixed time for the rotation matrix
    
    #calculate the rdots. need to build the pose matrices
    deltaf = measurment(r0_scaled, pose1, pose2, pose3, pose4, time)
    
    pose1_rdot1 = [pose1 deltaf[1]]
    pose2_rdot2 = [pose2 deltaf[2]]
    pose3_rdot3 = [pose3 deltaf[3]]
    pose4_rdot4 = [pose4 deltaf[4]]
    
    all_sats_scaled = vcat(pose1_rdot1,pose2_rdot2,pose3_rdot3,pose4_rdot4)
    
    #Generate a guess at the centroid of all 4 sats on the surface of the Earth. Scaled to km
    
    centroid_guess = [(eci_1[1,i]+eci_2[1,i]+eci_3[1,i]+eci_4[1,i])/4, (eci_1[2,i]+eci_2[2,i]+eci_3[2,i]+eci_4[2,i])/4, (eci_1[3,i]+eci_2[3,i]+eci_3[3,i]+eci_4[3,i])/4] 
    onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
    geodetic = [onearth[1], onearth[2], 0]

    xyz_guess = sGEOCtoECEF(geodetic, use_degrees = true)*1e-3 #change to km

    xyz_guess
    
    #Find all possible positions from montecarlo and send unscaled guess
    mean_rescaled, all_r0, iters = receiver_pose(all_sats_scaled, xyz_guess, zenith_angles, time, zdot)
    
    all_r0_scaled = zeros(n,4)

    #Rescale back to units (m and s)
    for i in 1:n
    
        all_r0_scaled[i,:] = [all_r0[i][1:3]*distance_scale*1e3; all_r0[i][4]*frequency_scale]
        
    end
    
    totalsum = zeros(4,4)

    for i in 1:n
        value = all_r0_scaled[i,:] - mean_rescaled
        matrixvalue = value*transpose(value)
        totalsum += matrixvalue
    
    end
    
    covariancematrix = totalsum/n
    
    PDOP = sqrt(covariancematrix[1,1] + covariancematrix[2,2] + covariancematrix[3,3])
    PDOP_array[i,1] = PDOP
    println("cov position erors: ", PDOP)
    println("Mean (m): ", mean_rescaled)
    println("Iterations: ", iters)
    println("Timestep: ", i)

end    

cov position erors: 9615.095870370289
Mean (m): [-6.128962678666212e6, -1.5889175933740225e6, 776222.0893315725, 3.99526501529268e8]
Iterations: 4
Timestep: 203
cov position erors: 9662.253041964766
Mean (m): [-6.128883258509191e6, -1.589446321181393e6, 776186.7002979694, 3.995038804398183e8]
Iterations: 4
Timestep: 204
cov position erors: 9331.083665193246
Mean (m): [-6.129126814443074e6, -1.588199718807765e6, 776270.1476555248, 3.9954639704893e8]
Iterations: 4
Timestep: 205
cov position erors: 9081.482727701949
Mean (m): [-6.128996030742024e6, -1.5890628940780465e6, 776213.3931949261, 3.9951052272370744e8]
Iterations: 4
Timestep: 206
cov position erors: 9225.564866798335
Mean (m): [-6.129048205807786e6, -1.5888657291313617e6, 776223.4989971414, 3.9951756821146125e8]
Iterations: 4
Timestep: 207
cov position erors: 9179.150594374976
Mean (m): [-6.12902038717555e6, -1.5892689156931194e6, 776200.3652562804, 3.994887132137843e8]
Iterations: 4
Timestep: 208
cov position erors: 9317.1291598

cov position erors: 8283.44512063521
Mean (m): [-6.12885913562694e6, -1.5900234757697822e6, 776155.5770171228, 3.994579229866711e8]
Iterations: 4
Timestep: 254
cov position erors: 8241.340052176434
Mean (m): [-6.129087959921982e6, -1.5894614868346516e6, 776199.332055024, 3.99459397927501e8]
Iterations: 4
Timestep: 255
cov position erors: 7950.656436918375
Mean (m): [-6.12908211634501e6, -1.589539266879681e6, 776198.3582458422, 3.994525158392714e8]
Iterations: 4
Timestep: 256
cov position erors: 8287.092249665035
Mean (m): [-6.12906844859798e6, -1.5896112940957972e6, 776197.4194345846, 3.9944477013431674e8]
Iterations: 4
Timestep: 257
cov position erors: 8054.212333254952
Mean (m): [-6.129130890835173e6, -1.589605823639685e6, 776202.2646445747, 3.994277798924606e8]
Iterations: 4
Timestep: 258
cov position erors: 8120.183753618369
Mean (m): [-6.129085803611737e6, -1.5896317123452416e6, 776195.8286587918, 3.9943731282640404e8]
Iterations: 4
Timestep: 259
cov position erors: 8324.118304082

cov position erors: 12235.899604249098
Mean (m): [-6.128621396975815e6, -1.5905706077172256e6, 776088.841641606, 3.994806276261594e8]
Iterations: 15
Timestep: 305
cov position erors: 11008.99025639883
Mean (m): [-6.128714317068052e6, -1.5903593829012818e6, 776128.0342789879, 3.994762668945886e8]
Iterations: 14
Timestep: 306
cov position erors: 10946.285367625838
Mean (m): [-6.128553244123903e6, -1.5906364384314362e6, 776070.6101081802, 3.994898724943132e8]
Iterations: 14
Timestep: 307
cov position erors: 10485.064234196363
Mean (m): [-6.128672762310784e6, -1.5902769222451302e6, 776111.9791970244, 3.994880144395348e8]
Iterations: 14
Timestep: 308
cov position erors: 10450.87651900654
Mean (m): [-6.128667562421517e6, -1.5903132665912411e6, 776109.4109667047, 3.9948605121907544e8]
Iterations: 14
Timestep: 309
cov position erors: 9754.023981994585
Mean (m): [-6.128889379973701e6, -1.5900484793573124e6, 776157.7321917871, 3.9944777025722384e8]
Iterations: 13
Timestep: 310
cov position erors

cov position erors: 22309.256811402887
Mean (m): [-6.12904554267402e6, -1.5903209754977191e6, 776161.2822876528, 3.9943267848488206e8]
Iterations: 4
Timestep: 356
cov position erors: 25951.704126711742
Mean (m): [-6.12918353899053e6, -1.5885829449705307e6, 776308.1524404273, 3.995646844751271e8]
Iterations: 12
Timestep: 357
cov position erors: 32390.059939561914
Mean (m): [-6.129235512906516e6, -1.5902832432242245e6, 776189.6914844981, 3.99463777774802e8]
Iterations: 5
Timestep: 358
cov position erors: 40985.55849968769
Mean (m): [-6.129862981621019e6, -1.5878946755156682e6, 776423.7435077023, 3.9956920578145415e8]
Iterations: 13
Timestep: 359
cov position erors: 59448.0818682739
Mean (m): [-6.130752398344042e6, -1.5886731813452279e6, 776465.3983920771, 3.995276226471112e8]
Iterations: 5
Timestep: 360
cov position erors: 109188.36726422099
Mean (m): [-6.135134318893509e6, -1.5878555084403309e6, 777013.6446463721, 3.995638326254506e8]
Iterations: 5
Timestep: 361
cov position erors: 3357

cov position erors: 7434.488436713999
Mean (m): [-6.128589102660818e6, -1.591080382418447e6, 776101.3955404295, 3.994529325474784e8]
Iterations: 4
Timestep: 407
cov position erors: 7372.597464282282
Mean (m): [-6.1287700800427e6, -1.5904049306477108e6, 776137.1941192598, 3.9944635429905814e8]
Iterations: 4
Timestep: 408
cov position erors: 7950.930723553019
Mean (m): [-6.128650123401158e6, -1.590771376749427e6, 776117.7488474124, 3.994588806375424e8]
Iterations: 4
Timestep: 409
cov position erors: 7676.062925148201
Mean (m): [-6.12877230771962e6, -1.5904491347664443e6, 776136.2144656426, 3.9945144506621784e8]
Iterations: 4
Timestep: 410
cov position erors: 7605.649962834687
Mean (m): [-6.128706625112535e6, -1.5905489394142618e6, 776127.3021973886, 3.994574444716533e8]
Iterations: 4
Timestep: 411
cov position erors: 7389.691202581132
Mean (m): [-6.128625782728021e6, -1.5908081891584457e6, 776110.7474469857, 3.9945898950382996e8]
Iterations: 4
Timestep: 412
cov position erors: 7887.71846

In [72]:
final_PDOP = PDOP_array[PDOP_array .!=0]

PDOP_max = findmax(final_PDOP)[1]
PDOP_min = findmin(final_PDOP)[1]

println("Maximum PDOP: ", PDOP_max)
println("Minimum PDOP: ", PDOP_min)

Maximum PDOP: 335704.8842467286
Minimum PDOP: 1813.5246806381742


In [74]:
PDOP_min = findmin(final_PDOP)

(1813.5246806381742, 98)

In [79]:
final_PDOP[99]

15698.709785223993

In [154]:
r0_scaled

4-element Vector{Float64}:
 -9609.08282878809
 -2493.214201145247
  1216.8918371970192
     4.0e6

In [155]:
initial_guess = [-9625.008623511323, -2580.926606494148, 835.4931473148392, 3.9e6]

4-element Vector{Float64}:
 -9625.008623511323
 -2580.926606494148
   835.4931473148392
     3.9e6

In [156]:
r0_scaled - initial_guess

4-element Vector{Float64}:
     15.925794723232684
     87.71240534890103
    381.39868988217995
 100000.0

In [22]:
actualpose =[-6.128804e6, -1.590206e6, 776.1502e3, 1e6]

4-element Vector{Float64}:
     -6.128804e6
     -1.590206e6
 776150.2
      1.0e6

In [382]:
log10(79.34904895113688)

1.8995417258262355